In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("Fast Food Analysis Project Spreadsheet.xlsx")

In [3]:
df.head()

,Unnamed: 0,Restaurant,Item Name,Calories,Protein (g),Total Fats (g),Sodium (mg),Total Sugars (g),Cholestrol (mg),Price ($),Carbs (g),Fiber (g)
0,NaN,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2
1,NaN,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2
2,NaN,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2
3,NaN,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3
4,NaN,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0


In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df.head()

,Restaurant,Item Name,Calories,Protein (g),Total Fats (g),Sodium (mg),Total Sugars (g),Cholestrol (mg),Price ($),Carbs (g),Fiber (g)
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0


In [6]:
df.shape

(49, 11)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Restaurant        49 non-null     object 
 1   Item Name         49 non-null     object 
 2   Calories          49 non-null     int64  
 3   Protein (g)       49 non-null     int64  
 4   Total Fats (g)    49 non-null     int64  
 5   Sodium (mg)       49 non-null     int64  
 6   Total Sugars (g)  49 non-null     int64  
 7   Cholestrol (mg)   49 non-null     int64  
 8   Price ($)         49 non-null     float64
 9   Carbs (g)         49 non-null     int64  
 10  Fiber (g)         49 non-null     int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 4.3+ KB


In [8]:
df.isnull().sum()


Restaurant          0
Item Name           0
Calories            0
Protein (g)         0
Total Fats (g)      0
Sodium (mg)         0
Total Sugars (g)    0
Cholestrol (mg)     0
Price ($)           0
Carbs (g)           0
Fiber (g)           0
dtype: int64

In [ ]:
df.